In [1]:
import os
import glob
import pandas as pd
from PIL import Image

In [2]:
input_dir = "./image"
output_train_dir = "./train"
output_test_dir = "./test"

In [3]:
# label
labels = pd.read_csv("label.csv")

# augumentation setting
alpha = labels["like"].mean()
times = int((1-alpha)/alpha)  # 正例が少ないものとして比率を算出
n = 300  # 水増し画像は(n,n)で加工したのち(224,224)として出力される
dig = 15  # 水増し画像は(-dig,dig)の範囲で回転加工される
idigre = dig*2/(times-1)  # 水増し画像の回転角度

In [4]:
# outputディレクトリの指定
path_list = [f"{output_train_dir}/00_notlike/",
             f"{output_train_dir}/01_like/",
             f"{output_test_dir}/"]

# フォルダを作成
for path in path_list:
    if not os.path.exists(path):
        os.makedirs(path)

In [5]:
# 加工済ファイル出力
# train
for i, l in zip(labels["id"], labels["like"]):
    img = Image.open(f'{input_dir}/{i}.png')
    img = img.convert("RGB")
    w, h = img.size
    if w/h > 1:
        img = img.crop((w/6, 0, w/2, h))
        w = w/2
    if (times > 1) & (l == 1):
        for j in range(times):
            digree = idigre*j - dig
            img.rotate(digree).resize((n, n)).crop(
                ((n-224)/2, (n-224)/2, n-(n-224)/2, n-(n-224)/2)
            ).save(path_list[int(l)] + f"{i}.{j}.png")
    img = img.resize((224, 224))
    img.save(path_list[int(l)] + f"{i}.png")

In [6]:
# test id 取得
exist = pd.DataFrame({
    "png": labels["id"].apply(lambda x: f"{input_dir}/{x}.png"),
    "exist": "1"
}).drop_duplicates()
allpng = pd.DataFrame({
    "png": glob.glob(f"{input_dir}/*")
}).drop_duplicates().merge(exist, on="png", how="left")
test_id = list(allpng[allpng["exist"].isnull()]["png"].apply(
    lambda x: int(x.split(".png")[0].split("/")[-1])))

In [7]:
# 加工済ファイル出力
# test
for i in test_id:
    img = Image.open(f'{input_dir}/{i}.png')
    img = img.convert("RGB")
    w, h = img.size
    if w/h > 1:
        img = img.crop((w/6, 0, w/2, h))
        w = w/2
    img = img.resize((224, 224))
    img.save(path_list[2] + f"{i}.png")